In [1]:
import torch
from langchain.llms import HuggingFacePipeline
from langchain.agents import Tool, AgentExecutor, ConversationalChatAgent
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.agents import ConversationalAgent
from langchain.memory import ConversationBufferMemory

# Load the model and create a pipeline

In [2]:
model_name = "lightblue/suzume-llama-3-8B-multilingual"  # Replace with Llama 3 when available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Create the LangChain LLM

In [3]:
llm = HuggingFacePipeline(pipeline=pipe)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


# Create LLMChain with Step-by-Step Reasoning Prompt


NameError: name 'tools' is not defined

In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = """
Question: {question}
Think step by step:
1) Do I know the answer to this question?
2) If yes, what is the answer?
3) If no, what additional information do I need?

Answer: Let's approach this step by step:
"""

prompt = PromptTemplate(template=template, input_variables=["question"])
# chain = LLMChain(llm=llm, prompt=prompt)


# Define dummy-tool and other helpers

In [6]:
def search_database(query):
    # Simulated database search
    return f"Results for '{query}': ..."

tools = [
    Tool(
        name="Database Search",
        func=search_database,
        description="Useful for searching information in the database"
    )
]

# Set up conversational memory for the agent
memory = ConversationBufferMemory(memory_key="chat_history")
# Create the agent with memory
agent = ConversationalAgent.from_llm_and_tools(llm, tools, memory=memory)

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

In [7]:
def cannot_answer(question):
    forbidden_topics = ["politics", "religion", "personal information"]
    return any(topic in question.lower() for topic in forbidden_topics)

def agent_response(question):
    if cannot_answer(question):
        return "I'm sorry, but I'm not able to answer questions on this topic."
    else:
        return agent_executor.run(question)



# Test everything

In [8]:
questions = [
    "What is the capital of France?",
    "How does photosynthesis work?",
    "What are your political views?",
    "Can you search the database for information about climate change?",
]


for question in questions:
    print(f"Question: {question}")
    response = agent_response(question)
    print(f"Response: {response}\n")

Question: What is the capital of France?


> Entering new AgentExecutor chain...


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 